# Funciones para trading

3 de octubre de 2025

In [ ]:
import yfinance as yf  # Para descargar históricos de acciones de Yahoo Finance
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
#funcion para carga de datos (Imanol y Enrique)
def cargar_datos(ticker, fecha_inicio, fecha_final):
    ''' Recibe ticker, fecha de inicio y fin.
        Devuelve un DataFrame con los datos históricos descargados desde Yahoo Finance.
    '''
    datos = yf.download(ticker, start = fecha_inicio, end = fecha_final)
    datos.columns = datos.columns.droplevel(1)
    return datos

In [ ]:
# Función de estrategia SMA (Bruno y Emil)
def SMA(df, short, long):
    df['SMA_short'] = df['Close'].rolling(window=short).mean()
    df['SMA_long'] = df['Close'].rolling(window=long).mean()
    # Se definen las señalesDefine signals
    df['signal'] = 0  # Se crea una columna con valor de 0
    df.loc[df['SMA_short'] > df['SMA_long'], 'signal'] = 1  # Señal de compra
    df.loc[df['SMA_short'] <= df['SMA_long'], 'signal'] = -1  # Señal de venta
    return df[['Close','signal', 'Date']]

In [ ]:
# Función de estrategia de Bollinger (Jannete y Ale)
def bollinger(datos, long_window):
    datos['media_movil'] = datos['Close'].rolling(window = long_window).mean()
    datos['desviacion_movil'] = datos['Close'].rolling(window = long_window).std()
    datos['banda_superior'] = datos['media_movil'] + (2 * datos['desviacion_movil'])
    datos['banda_inferior'] = datos['media_movil'] - (2 * datos['desviacion_movil'])
    signal = np.where((datos["Close"] <= datos["banda_inferior"]), 1, -1)
    #signal = np.where((datos["Close"] <= datos["banda_inferior"]) & (datos["Close"] > datos["Close"].shift(1)), 1, -1)
    datos["signal"] = np.where(datos["Close"].isnull(), 0, signal)
    # Se eliminan los NAs
    datos = datos.dropna().copy()
    # Se genera una columna de posiciones, se recorren las señales para evitar
    # El sesgo look-akead
    # La posición refleja la señal del día anterior
    datos['posicion'] = datos['signal'].shift(1)

    # Se calcula el cambio porcentual diario del precio de la acción
    datos['rendim_diario'] = datos['Close'].pct_change()

    datos['rendim_estrategia'] = datos['posicion'] * datos['rendim_diario']

    # Se obtienen los rendimientos acumulados
    datos['rendim_acumulado_accion'] = (1 + datos['rendim_diario']).cumprod()
    datos['rendim_acumlado_estrategia'] = (1 + datos['rendim_estrategia']).cumprod()

    datos['max_acumulado'] = datos['rendim_acumlado_estrategia'].cummax()
    return datos

In [ ]:
# Función de estrategia de RSI (Imanol y Enrique)
def calcular_rsi(datos, n_rsi=14):
    '''
    Calcula el RSI y genera señales numéricas (1, -1, 0).
    Devuelve un DataFrame con las columnas 'Close' y 'signal'.
    '''


    if isinstance(datos['Close'], pd.DataFrame):
        close_series = datos['Close'].iloc[:, 0]
    else:
        close_series = datos['Close']

    datos['cambio'] = close_series.diff()

    datos['gain'] = datos['cambio'].apply(lambda x: x if x > 0 else 0)
    datos['loss'] = datos['cambio'].apply(lambda x: abs(x) if x < 0 else 0)

    datos['avg_gain'] = datos['gain'].ewm(span = n_rsi, adjust=False).mean()
    datos['avg_loss'] = datos['loss'].ewm(span = n_rsi, adjust=False).mean()

    datos['rs'] = datos['avg_gain'] / datos['avg_loss']

    datos['rsi'] = 100 - (100 / (1 + datos['rs']))

    signal = np.where((datos['rsi'] < 30), 1, 0)
    signal = np.where((datos['rsi'] > 70), -1, signal)

    datos["signal"] = np.where(close_series.isnull(), 0, signal)

    return datos[['Close', 'signal']]

In [ ]:
def estrategia_sma_ema(datos, short_window, long_window=200):
  # Ventana
  datos['SMA'] = datos['Close'].rolling(window = long_window).mean()
  datos['EMA'] = datos['Close'].ewm(span = long_window, adjust=False).mean()

  # Construimos la regla
  signal = np.where((datos["Close"] > datos["SMA"]) & (datos["Close"] > datos["EMA"]), 1, -1)
  datos["signal"] = np.where(datos["Close"].isnull(), 0, signal)

  # Se eliminan los NAs
  return datos.dropna().copy()


def calculo_rendims_sma_ema(datos):
  # Se genera una columna de posiciones, se recorren las señales para evitar
  # El sesgo look-akead
  # La posición refleja la señal del día anterior
  datos['posicion'] = datos['signal'].shift(1)

  # Se calcula el cambio porcentual diario del precio de la acción
  datos['rendim_diario'] = datos['Close'].pct_change()

  datos['rendim_estrategia'] = datos['posicion'] * datos['rendim_diario']
  return datos


def calcula_descriptivos(datos):
    resumen = {}

    rend_med_anual = datos[['rendim_diario', 'rendim_estrategia']].mean() * 252
    tasa_anual = np.exp(rend_med_anual) - 1
    vol_anual = datos[['rendim_diario', 'rendim_estrategia']].std() * np.sqrt(252)

    resumen['rend_med_anual'] = rend_med_anual
    resumen['tasa_anual'] = tasa_anual
    resumen['vol_anual'] = vol_anual

    # También podrías añadir Sharpe, etc.
    sharpe = rend_med_anual / vol_anual
    resumen['sharpe'] = sharpe

    return resumen



def calcula_max_acumulado(datos):
  datos['rendim_acumulado_accion'] = (1 + datos['rendim_diario']).cumprod()
  datos['rendim_acumulado_estrategia'] = (1 + datos['rendim_estrategia']).cumprod()

  datos['max_acumulado'] = datos['rendim_acumulado_estrategia'].cummax()
  return datos.dropna()